<a href="https://colab.research.google.com/github/LPDP22/PIX_MuncipioAtiv/blob/main/Pix_Municipio_Atividade_com_Java_e_Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
url = "https://olinda.bcb.gov.br/olinda/servico/Pix_DadosAbertos/versao/v1/odata/TransacoesPixPorMunicipio(DataBase=@DataBase)?@DataBase='202412'&$top=100&$format=json&$select=VL_PagadorPF,QT_PagadorPF,VL_PagadorPJ,QT_PagadorPJ,VL_RecebedorPF,QT_RecebedorPF,VL_RecebedorPJ,QT_RecebedorPJ,QT_PES_PagadorPF,QT_PES_PagadorPJ,QT_PES_RecebedorPF,QT_PES_RecebedorPJ"

In [2]:
import requests
import json

Criar uma função para obter os dados:

In [14]:

def pix_municipio(link):
  resposta = requests.get(link)

  if resposta.status_code == 200:
    dados = resposta.json()
    print('Status Code:', resposta.status_code)
    with open('resultado.json', 'w', encoding='utf-8') as arquivo:
      json.dump(dados, arquivo, ensure_ascii=False, indent=4)

  else:
    print('Status Code:', resposta.status_code)


In [4]:
pix_municipio(url)

Status Code: 200


In [10]:
with open("resultado.json", "r", encoding="utf-8") as f:
    dados = json.load(f)

registros = dados["value"]


In [6]:
print(registros)

[{'VL_PagadorPF': 34711492.5, 'QT_PagadorPF': 270041, 'VL_PagadorPJ': 8618857.05, 'QT_PagadorPJ': 7906, 'VL_RecebedorPF': 32539920.25, 'QT_RecebedorPF': 158291, 'VL_RecebedorPJ': 7438821.4, 'QT_RecebedorPJ': 36623, 'QT_PES_PagadorPF': 5805, 'QT_PES_PagadorPJ': 119, 'QT_PES_RecebedorPF': 5580, 'QT_PES_RecebedorPJ': 113}, {'VL_PagadorPF': 3423227.59, 'QT_PagadorPF': 22126, 'VL_PagadorPJ': 703213.62, 'QT_PagadorPJ': 816, 'VL_RecebedorPF': 3037716.09, 'QT_RecebedorPF': 13039, 'VL_RecebedorPJ': 720597.18, 'QT_RecebedorPJ': 2013, 'QT_PES_PagadorPF': 587, 'QT_PES_PagadorPJ': 36, 'QT_PES_RecebedorPF': 571, 'QT_PES_RecebedorPJ': 34}, {'VL_PagadorPF': 69411271.97, 'QT_PagadorPF': 614783, 'VL_PagadorPJ': 6221032.09, 'QT_PagadorPJ': 7990, 'VL_RecebedorPF': 65429212.35, 'QT_RecebedorPF': 385406, 'VL_RecebedorPJ': 6038963.54, 'QT_RecebedorPJ': 43716, 'QT_PES_PagadorPF': 15081, 'QT_PES_PagadorPJ': 264, 'QT_PES_RecebedorPF': 14984, 'QT_PES_RecebedorPJ': 232}, {'VL_PagadorPF': 28520204.1, 'QT_PagadorPF

In [7]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-11-jdk -qq > /dev/null
!pip install -q pyspark

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [8]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["PATH"] += os.pathsep + os.path.join(os.environ["JAVA_HOME"], "bin")

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round

spark = SparkSession.builder \
    .appName("BCB_API_PySpark") \
    .master("local[*]") \
    .getOrCreate()


In [12]:
df = spark.createDataFrame(registros)

df.printSchema()

root
 |-- QT_PES_PagadorPF: long (nullable = true)
 |-- QT_PES_PagadorPJ: long (nullable = true)
 |-- QT_PES_RecebedorPF: long (nullable = true)
 |-- QT_PES_RecebedorPJ: long (nullable = true)
 |-- QT_PagadorPF: long (nullable = true)
 |-- QT_PagadorPJ: long (nullable = true)
 |-- QT_RecebedorPF: long (nullable = true)
 |-- QT_RecebedorPJ: long (nullable = true)
 |-- VL_PagadorPF: double (nullable = true)
 |-- VL_PagadorPJ: double (nullable = true)
 |-- VL_RecebedorPF: double (nullable = true)
 |-- VL_RecebedorPJ: double (nullable = true)



In [13]:
df.show(5, truncate=False)

+----------------+----------------+------------------+------------------+------------+------------+--------------+--------------+-------------+-------------+--------------+--------------+
|QT_PES_PagadorPF|QT_PES_PagadorPJ|QT_PES_RecebedorPF|QT_PES_RecebedorPJ|QT_PagadorPF|QT_PagadorPJ|QT_RecebedorPF|QT_RecebedorPJ|VL_PagadorPF |VL_PagadorPJ |VL_RecebedorPF|VL_RecebedorPJ|
+----------------+----------------+------------------+------------------+------------+------------+--------------+--------------+-------------+-------------+--------------+--------------+
|5805            |119             |5580              |113               |270041      |7906        |158291        |36623         |3.47114925E7 |8618857.05   |3.253992025E7 |7438821.4     |
|587             |36              |571               |34                |22126       |816         |13039         |2013          |3423227.59   |703213.62    |3037716.09    |720597.18     |
|15081           |264             |14984             |232   

In [17]:
from pyspark.sql import functions as F

In [15]:
# 1. Selecionar colunas específicas
df_select = df.select("QT_PES_PagadorPF", "QT_PES_RecebedorPF")
df_select.show(5)

+----------------+------------------+
|QT_PES_PagadorPF|QT_PES_RecebedorPF|
+----------------+------------------+
|            5805|              5580|
|             587|               571|
|           15081|             14984|
|            2800|              2793|
|            2658|              2524|
+----------------+------------------+
only showing top 5 rows



In [19]:
# 2. Criar novas colunas
df_novo = df.withColumn("PF_total", F.col("QT_PES_PagadorPF") + F.col("QT_PES_RecebedorPF"))
df_novo.show(5)

+----------------+----------------+------------------+------------------+------------+------------+--------------+--------------+-------------+-------------+--------------+--------------+--------+
|QT_PES_PagadorPF|QT_PES_PagadorPJ|QT_PES_RecebedorPF|QT_PES_RecebedorPJ|QT_PagadorPF|QT_PagadorPJ|QT_RecebedorPF|QT_RecebedorPJ| VL_PagadorPF| VL_PagadorPJ|VL_RecebedorPF|VL_RecebedorPJ|PF_total|
+----------------+----------------+------------------+------------------+------------+------------+--------------+--------------+-------------+-------------+--------------+--------------+--------+
|            5805|             119|              5580|               113|      270041|        7906|        158291|         36623| 3.47114925E7|   8618857.05| 3.253992025E7|     7438821.4|   11385|
|             587|              36|               571|                34|       22126|         816|         13039|          2013|   3423227.59|    703213.62|    3037716.09|     720597.18|    1158|
|           150

In [20]:
# 3. Filtrar registros (condição)
df_filtrado = df.filter(F.col("QT_PES_PagadorPF") > 100_000)
df_filtrado.show(5)

+----------------+----------------+------------------+------------------+------------+------------+--------------+--------------+-----------------+------------------+-----------------+-----------------+
|QT_PES_PagadorPF|QT_PES_PagadorPJ|QT_PES_RecebedorPF|QT_PES_RecebedorPJ|QT_PagadorPF|QT_PagadorPJ|QT_RecebedorPF|QT_RecebedorPJ|     VL_PagadorPF|      VL_PagadorPJ|   VL_RecebedorPF|   VL_RecebedorPJ|
+----------------+----------------+------------------+------------------+------------+------------+--------------+--------------+-----------------+------------------+-----------------+-----------------+
|         4729426|          347461|           4600828|            324097|   163669090|    30504953|     107949461|     143608287|3.203895416135E10|1.0237497502473E11|3.122338089549E10|8.923373029144E10|
|          157630|           10311|            157437|              9613|     6065792|      340087|       3865685|        994160|   8.9862816788E8|    6.0196882605E8|   9.2992339224E8|   5

In [21]:
# 4. Ordenar valores
df_ordenado = df.orderBy(F.col("QT_PES_PagadorPF").desc())
df_ordenado.show(10)

+----------------+----------------+------------------+------------------+------------+------------+--------------+--------------+-----------------+------------------+-----------------+-----------------+
|QT_PES_PagadorPF|QT_PES_PagadorPJ|QT_PES_RecebedorPF|QT_PES_RecebedorPJ|QT_PagadorPF|QT_PagadorPJ|QT_RecebedorPF|QT_RecebedorPJ|     VL_PagadorPF|      VL_PagadorPJ|   VL_RecebedorPF|   VL_RecebedorPJ|
+----------------+----------------+------------------+------------------+------------+------------+--------------+--------------+-----------------+------------------+-----------------+-----------------+
|         4729426|          347461|           4600828|            324097|   163669090|    30504953|     107949461|     143608287|3.203895416135E10|1.0237497502473E11|3.122338089549E10|8.923373029144E10|
|          269311|           26973|            217693|             24360|     1627730|      180309|        902613|        438267|   4.8014994483E8|    6.1876313099E8|    5.086309116E8|   6

In [22]:
# 5. Selecionar e renomear colunas
df_renomeado = df.select(
    F.col("QT_PES_PagadorPF").alias("qtpes_pagPF"),
    F.col("QT_PES_RecebedorPF").alias("qtpes_recPF")
)
df_renomeado.show(5)

+-----------+-----------+
|qtpes_pagPF|qtpes_recPF|
+-----------+-----------+
|       5805|       5580|
|        587|        571|
|      15081|      14984|
|       2800|       2793|
|       2658|       2524|
+-----------+-----------+
only showing top 5 rows



**Ações**: Executam o plano de transformação e retornam resultados

In [23]:
# 1. Mostrar registros
df.show(5)

+----------------+----------------+------------------+------------------+------------+------------+--------------+--------------+-------------+-------------+--------------+--------------+
|QT_PES_PagadorPF|QT_PES_PagadorPJ|QT_PES_RecebedorPF|QT_PES_RecebedorPJ|QT_PagadorPF|QT_PagadorPJ|QT_RecebedorPF|QT_RecebedorPJ| VL_PagadorPF| VL_PagadorPJ|VL_RecebedorPF|VL_RecebedorPJ|
+----------------+----------------+------------------+------------------+------------+------------+--------------+--------------+-------------+-------------+--------------+--------------+
|            5805|             119|              5580|               113|      270041|        7906|        158291|         36623| 3.47114925E7|   8618857.05| 3.253992025E7|     7438821.4|
|             587|              36|               571|                34|       22126|         816|         13039|          2013|   3423227.59|    703213.62|    3037716.09|     720597.18|
|           15081|             264|             14984|      

In [24]:
# 2. Contar número total de registros
print("Total de registros:", df.count())

Total de registros: 100


In [25]:
# 3. Descrever estatísticas básicas
df.describe().show()

+-------+-----------------+------------------+------------------+------------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+
|summary| QT_PES_PagadorPF|  QT_PES_PagadorPJ|QT_PES_RecebedorPF|QT_PES_RecebedorPJ|        QT_PagadorPF|      QT_PagadorPJ|      QT_RecebedorPF|      QT_RecebedorPJ|        VL_PagadorPF|        VL_PagadorPJ|     VL_RecebedorPF|     VL_RecebedorPJ|
+-------+-----------------+------------------+------------------+------------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+
|  count|              100|               100|               100|               100|                 100|               100|                 100|                 100|                 100|                 100|                100|                100|
|   

In [27]:
# 4. Calcular médias de colunas
df.select(
    F.mean("QT_PES_PagadorPF").alias("med_pespagPF"),
    F.mean("QT_PagadorPJ").alias("med_pagPJ")
).show()

+------------+---------+
|med_pespagPF|med_pagPJ|
+------------+---------+
|    74594.62|376630.51|
+------------+---------+



In [28]:
# 5. Obter o registro com maior valorPix
df.orderBy(F.col("QT_PES_PagadorPJ").desc()).limit(6).show()

+----------------+----------------+------------------+------------------+------------+------------+--------------+--------------+-----------------+------------------+-----------------+-----------------+
|QT_PES_PagadorPF|QT_PES_PagadorPJ|QT_PES_RecebedorPF|QT_PES_RecebedorPJ|QT_PagadorPF|QT_PagadorPJ|QT_RecebedorPF|QT_RecebedorPJ|     VL_PagadorPF|      VL_PagadorPJ|   VL_RecebedorPF|   VL_RecebedorPJ|
+----------------+----------------+------------------+------------------+------------+------------+--------------+--------------+-----------------+------------------+-----------------+-----------------+
|         4729426|          347461|           4600828|            324097|   163669090|    30504953|     107949461|     143608287|3.203895416135E10|1.0237497502473E11|3.122338089549E10|8.923373029144E10|
|          260064|           33659|            250116|             30458|     8651518|     1570916|       4964750|       4036632|  2.23184003901E9|   7.51815142535E9|  2.22356027453E9|  7.